In [1]:
import os
try:
    from yaml import load, CLoader as Loader
except:
    from yaml import load, Loader

data = load(open('env.yaml'), Loader=Loader)
os.environ.update({key: str(val) for key, val in data.items()})
os.environ['DATABASE_URL'] = 'sqlite:///:memory:'

from hemlock import push_app_context

app = push_app_context()

In [2]:
from hemlock import Page, Label

Page(Label('<p>Hello, World!</p>')).preview()

<Page 2>

In [3]:
[os.remove(f) for f in app.tmpfiles if os.path.exists(f)]

[None]

In [1]:
import pandas as pd

df = pd.read_csv('data.csv')
df.head()

,Test,IPv4,Ideal,WorkerId,SurveyCode,ID,EndTime,StartTime,Status,DemographicsTime,...,RaceBlack,RaceSouth Asian,RaceEast Asian,RaceArabic or Central Asian,RaceOther,RaceSpecify,Male,Age,ArticleName,Enjoy
0,1,127.0.0.1,0,test,s7X1T6,1,2021-01-04 17:30:49.562080,2021-01-04 17:30:38.818873,Completed,4.191885,...,0,1,0,0,0,NaN,1,51.260780,Iran-tanker,7.0
1,1,127.0.0.1,0,test,s7X1T6,1,2021-01-04 17:30:49.562080,2021-01-04 17:30:38.818873,Completed,4.191885,...,0,1,0,0,0,NaN,1,51.260780,UK-extradite-wikileaks,2.0
2,1,127.0.0.1,1,test,D95Lvq,2,2021-01-04 17:31:08.415142,2021-01-04 17:30:55.928005,Completed,4.414473,...,0,0,0,0,0,NaN,0,4.008214,Iran-tanker,7.0
3,1,127.0.0.1,1,test,D95Lvq,2,2021-01-04 17:31:08.415142,2021-01-04 17:30:55.928005,Completed,4.414473,...,0,0,0,0,0,NaN,0,4.008214,UK-extradite-wikileaks,9.0
4,1,127.0.0.1,0,test,nOOfm6,3,2021-01-04 17:31:28.172381,2021-01-04 17:31:14.798670,Completed,4.517570,...,0,0,0,1,0,NaN,1,63.342916,UK-extradite-wikileaks,4.0


In [2]:
df.columns

Index(['Test', 'IPv4', 'Ideal', 'WorkerId', 'SurveyCode', 'ID', 'EndTime',
       'StartTime', 'Status', 'DemographicsTime', 'Gender', 'GenderSpecify',
       'BirthMonth', 'RaceWhite', 'RaceBlack', 'RaceSouth Asian',
       'RaceEast Asian', 'RaceArabic or Central Asian', 'RaceOther',
       'RaceSpecify', 'Male', 'Age', 'ArticleName', 'Enjoy'],
      dtype='object')

In [3]:
cols = [
    'Ideal',
    'RaceWhite',
    'RaceBlack',
    'RaceSouth Asian',
    'RaceEast Asian',
    'RaceArabic or Central Asian',
    'RaceOther',
    'Male',
    'Age',
    'ArticleName'
]
X, y = df[cols], df.Enjoy
X.head()

,Ideal,RaceWhite,RaceBlack,RaceSouth Asian,RaceEast Asian,RaceArabic or Central Asian,RaceOther,Male,Age,ArticleName
0,0,0,0,1,0,0,0,1,51.260780,Iran-tanker
1,0,0,0,1,0,0,0,1,51.260780,UK-extradite-wikileaks
2,1,1,0,0,0,0,0,0,4.008214,Iran-tanker
3,1,1,0,0,0,0,0,0,4.008214,UK-extradite-wikileaks
4,0,0,0,0,0,1,0,1,63.342916,UK-extradite-wikileaks


In [4]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline

reg = make_pipeline(
    make_column_transformer(
        (OneHotEncoder(), ['ArticleName'])
    ),
    RandomForestRegressor()
)
reg.fit(X, y)
reg.score(X, y)

0.0007860611335934742

In [5]:
from sklearn.model_selection import cross_val_score

cross_val_score(reg, X, y).mean()

-1.6380992176362152

In [6]:
from joblib import dump

dump(reg, 'model.p')

['model.p']

In [7]:
df = pd.read_csv('user_data.csv')
df.head()

,Unnamed: 0,IPv4,Condition,WorkerId,ID,EndTime,StartTime,Status,DemographicsTime,Gender,...,BirthMonth,RaceWhite,RaceBlack,RaceSouth Asian,RaceEast Asian,RaceArabic or Central Asian,RaceOther,RaceSpecify,Male,Age
0,0,127.0.0.1,random,test,2,2021-01-04 17:46:04.605200,2021-01-04 17:45:49.176281,InProgress,10.06271,Male,...,1992-10-01 00:00:00,1,0,0,0,0,0,NaN,1,28.260096


In [8]:
from articles import ARTICLES, ARTICLE_NAMES

df['Ideal'] = df['Condition'] == 'ideal'
df = df.append((len(ARTICLES)-1)*[df], ignore_index=True)
df['ArticleName'] = ARTICLE_NAMES
X = df[cols]
X.head()

,Ideal,RaceWhite,RaceBlack,RaceSouth Asian,RaceEast Asian,RaceArabic or Central Asian,RaceOther,Male,Age,ArticleName
0,False,1,0,0,0,0,0,1,28.260096,GA-Trump
1,False,1,0,0,0,0,0,1,28.260096,Iran-tanker
2,False,1,0,0,0,0,0,1,28.260096,UK-extradite-wikileaks


In [9]:
y_pred = reg.predict(X)
article_ratings = list(zip(ARTICLES, y_pred))
article_ratings.sort(key=lambda x: x[1], reverse=True)
article_ratings

[(('UK-extradite-wikileaks',
   'British Court Rejects U.S. Request To Extradite WikiLeaks Founder Julian Assange',
   'https://www.npr.org/2021/01/04/953142687/british-court-rejects-u-s-request-to-extradite-wikileaks-founder-julian-assange'),
  4.878583333333333),
 (('Iran-tanker',
   'Iran seizes tanker, ramps up uranium enrichment in fresh escalation with West',
   'https://www.nbcnews.com/news/world/iran-resumes-20-percent-uranium-enrichment-nuclear-facility-state-news-n1252708'),
  4.801992063492062),
 (('GA-Trump',
   "Georgia GOP lieutenant governor says Trump call with secretary of state 'inappropriate'",
   'https://www.cnn.com/2021/01/04/politics/geoff-duncan-trump-raffensperger-call-cnntv/index.html'),
  4.47166045066045)]